# Credit Risk Prediction API
 
This notebook demonstrates:
1. API server setup
2. Model serving
3. Real-time prediction testing

## 1. Install Dependencies
Run this cell first

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [14]:
!pip install fastapi uvicorn python-multipart joblib



## 2. Verify Model File

In [15]:

import os
from pathlib import Path

# 1. Navigate to project root
project_root = Path().resolve().parent
os.chdir(project_root)  # Change working directory

# 2. Verify paths
print(f"Current Directory: {os.getcwd()}")
print(f"API File Exists: {Path('scripts/api.py').exists()}")



Current Directory: C:\Users\Tsebaot\Desktop\week 6\credit-risk-modeling
API File Exists: True


In [16]:

MODEL_PATH = Path("models/credit_risk_model.pkl")
assert MODEL_PATH.exists(), "Model file not found!"
print(f"Model exists: {MODEL_PATH.stat().st_size/1024:.1f} KB")



Model exists: 164.8 KB


In [17]:
# Check if uvicorn is available
!uvicorn --version

In [18]:
# Check paths before starting server
from pathlib import Path

# Should print: credit-risk-modeling/models/credit_risk_model.pkl
print("Model path:", (Path().cwd() / "models" / "credit_risk_model.pkl").resolve())

Model path: C:\Users\Tsebaot\Desktop\week 6\credit-risk-modeling\models\credit_risk_model.pkl


In [49]:
import os
print("Current working directory:", os.getcwd())
# Should show project root, not notebooks/

Current working directory: C:\Users\Tsebaot\Desktop\week 6\credit-risk-modeling


## 3. Start API Server
Run this cell and keep it running

In [63]:

!uvicorn scripts.api:app --reload --host 0.0.0.0 --port 8000



## 4. Test API Endpoint
Run in a new cell while server is running

In [53]:
import requests

try:
    # Test model info endpoint
    response = requests.get("http://localhost:8000/model_info")
    print("Model Info:", response.json())
    
    # Test prediction endpoint
    test_data = {
        "Amount": 15000.0,
        "TransactionCount": 8,
        "Recency": 30
    }
    prediction = requests.post("http://localhost:8000/predict", json=test_data)
    print("Prediction:", prediction.json())
    
except Exception as e:
    print(f"Error: {str(e)}")
    print("Check the server logs above for detailed error messages!")

Model Info: {'detail': 'Not Found'}
Error: Expecting value: line 1 column 1 (char 0)
Check the server logs above for detailed error messages!


In [ ]:
import requests
print(requests.get("http://localhost:8000/model_info").json())

In [56]:
import requests


test_data = {
    "Amount": 15000.0,
    "TransactionCount": 8,
    "Recency": 30
}

try:
    response = requests.post("http://localhost:8000/predict", json=test_data)
    response.raise_for_status()  # Raise exception for 4XX/5XX status codes
    print("Prediction:", response.json())
except requests.exceptions.HTTPError as err:
    print(f"HTTP Error: {err}")
    print("Server Response:", err.response.text)
except Exception as err:
    print(f"Other Error: {err}")


#print("Response Status:", response.status_code)
#print("Prediction Result:", json.dumps(response.json(), indent=2))

HTTP Error: 500 Server Error: Internal Server Error for url: http://localhost:8000/predict
Server Response: Internal Server Error


## 5. Batch Testing

In [ ]:
test_cases = [
    {"Amount": 5000, "TransactionCount": 2, "Recency": 60},
    {"Amount": 25000, "TransactionCount": 15, "Recency": 7},
    {"Amount": -3000, "TransactionCount": 3, "Recency": 45}
]

for case in test_cases:
    response = requests.post("http://localhost:8000/predict", json=case)
    print(f"Input: {case} => Risk: {response.json()['risk_score']}")




## 6. API Documentation
Access these while server is running:
- [API Docs](http://localhost:8000/docs)
- [Redoc](http://localhost:8000/redoc)


## Deployment Notes
To run as production:
```bash
uvicorn scripts.api:app --host 0.0.0.0 --port 80
```